In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
    library(pROC)
}))


In [2]:
run_name <- '1_p_factor_v1'
phenotype <- 'HC382'

data_d <- file.path(
    '/oak/stanford/groups/mrivas/projects/PRS/private_output/20200908_PRS_map_test', 
    run_name, phenotype, '1_fit_w_val'
)
refit <- F

# output
p_val_f <- sprintf('AUC_diff_p.%s.tsv', phenotype)


In [3]:
# input and parameters

#data_d <- '/oak/stanford/groups/mrivas/projects/PRS/private_output/20200908_PRS_map_test'
phe_f <- '/scratch/groups/mrivas/ukbb24983/phenotypedata/master_phe/master.20200828.phe.zst'
covariates       <- c('age', 'sex', paste0('PC', 1:10))
refit_split_strs <- c('non_british_white', 'african', 's_asian', 'e_asian')
sscore_f             <- file.path(data_d, '__PHENOTYPE__.sscore.zst')
snpnet_BETAs_f       <- file.path(data_d, 'snpnet.tsv')
snpnet_covar_BETAs_f <- file.path(data_d, 'snpnet.covars.tsv')
family <- ifelse((startsWith(phenotype, 'INI') | startsWith(phenotype, 'QT_FC')), 'gaussian', 'binomial')

# output
eval_f <- file.path(data_d, 'snpnet.eval.tsv')
plot_f <- file.path(data_d, 'snpnet.plot.pdf')
percentile_f <- file.path(data_d, 'snpnet.percentile.tsv')


In [4]:
source('/oak/stanford/groups/mrivas/users/ytanigaw/repos/rivas-lab/snpnet/helpers/snpnet_misc.R')

# read the raw phenotype file
fread(
    cmd=paste(cat_or_zcat(phe_f), phe_f,  '|', 'sed -e "s/^#//g"'),
    select=c('FID', 'IID', 'split', covariates, phenotype),
    colClasses = c('FID'='character', 'IID'='character'),
    data.table=F
) -> phe_df

if(refit){
    phe_df %>%
    mutate(
        split = if_else(split %in% c('train', 'val'), 'train_val', split)
    ) -> phe_df
}

# read PRS and covariate-based score
phe_df %>%
compute_phe_score_df(
    phenotype,
    str_replace_all(sscore_f, '__PHENOTYPE__', phenotype),
    str_replace_all(snpnet_covar_BETAs_f, '__PHENOTYPE__', phenotype),
    covariates, family, refit_split_strs
) -> phe_score_df


Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


In [21]:
Sys.glob(file.path(data_d, 'results', '*.sscore.zst'), dirmark = FALSE) %>%
lapply(function(x){as.integer(str_replace_all(basename(x), '^snpnet.lambda|.sscore.zst$', ''))}) %>%
simplify() %>% sort() -> lambda_idxs 


In [43]:
lambda_idxs 

[1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
[26] 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50
[51] 51 52 53 54 55 56 57 58 59 60

In [5]:
lambda_idxs %>%
lapply(function(lambda_idx){
    file.path(data_d, 'results', sprintf('snpnet.lambda%d.sscore.zst', lambda_idx)) %>%
    read_PRS() %>%
    rename(!!sprintf('geno_%d', lambda_idx) := 'geno_score')
}) %>%
reduce(function(x, y){inner_join(x, y, by=c('FID', 'IID'))}) -> geno_score_intermediates_df


In [25]:
phe_score_df %>% 
left_join(geno_score_intermediates_df, by=c('FID', 'IID')) -> phe_score_intermediate_df


In [26]:
phe_score_intermediate_df %>%
filter(split == 'val') %>%
mutate(geno_covar_score = geno_score + covar_score) -> sdf


In [27]:
roc_tests <- list()


In [28]:
roc.test(
    roc(sdf$phe, sdf[['covar_score']], levels=c('control'=1, 'case'=2), direction='<'),
    roc(sdf$phe, sdf[['geno_covar_score']], levels=c('control'=1, 'case'=2), direction='<')
) -> roc_tests[['geno_covar_vs_covar']]


In [29]:
roc.test(
    roc(sdf$phe, sdf[['covar_score']], levels=c('control'=1, 'case'=2), direction='<'),
    roc(sdf$phe, sdf[['geno_1']], levels=c('control'=1, 'case'=2), direction='<')
) -> roc_tests[['geno_1_vs_covar']]


In [32]:
for(lambda_idx in lambda_idxs[lambda_idxs != 1]){
    roc.test(
        roc(sdf$phe, sdf[[sprintf('geno_%d', lambda_idx - 1)]], levels=c('control'=1, 'case'=2), direction='<'),
        roc(sdf$phe, sdf[[sprintf('geno_%d', lambda_idx    )]], levels=c('control'=1, 'case'=2), direction='<')
    ) -> roc_tests[[sprintf('geno_%d_vs_geno_%d', lambda_idx, lambda_idx - 1)]]
}


In [33]:
roc_p_vals <- data.frame(
    name = names(roc_tests),
    p = names(roc_tests) %>%
    lapply(function(k){
        (roc_tests[[k]])$p.value
    }) %>% simplify(),
    stringsAsFactors=F
) %>%
separate(name, c('score2', 'score1'), sep='_vs_', remove=F) %>%
select(name, score1, score2, p)


In [34]:
roc_p_vals %>%
rename('#name' = 'name') %>%
fwrite(p_val_f, sep='\t', na = "NA", quote=F)


In [35]:
roc_tests[['geno_covar_vs_covar']]


	DeLong's test for two correlated ROC curves

data:  roc(sdf$phe, sdf[["covar_score"]], levels = c(control = 1, case = 2),  and roc(sdf$phe, sdf[["geno_covar_score"]], levels = c(control = 1,     direction = "<") and     case = 2), direction = "<")
Z = -16.507, p-value < 2.2e-16
alternative hypothesis: true difference in AUC is not equal to 0
sample estimates:
AUC of roc1 AUC of roc2 
  0.5281458   0.6189194 


In [36]:
roc_p_vals %>%
tail()

,name,score1,score2,p
,<chr>,<chr>,<chr>,<dbl>
56,geno_55_vs_geno_54,geno_54,geno_55,0.01316363
57,geno_56_vs_geno_55,geno_55,geno_56,0.10500133
58,geno_57_vs_geno_56,geno_56,geno_57,0.17819255
59,geno_58_vs_geno_57,geno_57,geno_58,0.35657788
60,geno_59_vs_geno_58,geno_58,geno_59,0.76033081
61,geno_60_vs_geno_59,geno_59,geno_60,0.36920904


- We currently have `lambda idx = 58` as the best model
- `lambda idx = 55` may be the optimal

In [37]:
# evaluate the predictive performance
phe_score_intermediate_df %>%
eval_performance(
    phenotype,
    str_replace_all(snpnet_BETAs_f, '__PHENOTYPE__', phenotype),
    family
) -> eval_58_df


In [38]:
phe_score_intermediate_df %>%
select(-geno_score) %>%
rename(geno_score = geno_55) %>%
eval_performance(
    phenotype,
    file.path(data_d, 'results', 'snpnet.lambda55.tsv'),
    family
) -> eval_55_df


In [39]:
eval_58_df

phenotype_name,split,geno,covar,geno_covar,geno_delta,n_variables,case_n,control_n
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>
HC382,train,0.7731822,0.5387748,0.7722529,0.233478152,7595,31741,204250
HC382,test,0.6116628,0.5352619,0.6161877,0.080925727,7595,8934,58491
HC382,val,0.6153153,0.5281458,0.6189194,0.090773671,7595,4553,29160
HC382,non_british_white,0.6110596,0.5676678,0.6276089,0.059941082,7595,3243,21662
HC382,s_asian,0.5784973,0.5760666,0.6068172,0.030750520,7595,1171,6660
HC382,african,0.5530495,0.6095357,0.6150603,0.005524621,7595,871,5626
HC382,e_asian,0.4973209,0.5933069,0.5660849,-0.027222022,7595,166,1538


In [46]:
phe_score_intermediate_df %>%
select(-geno_score) %>%
rename(geno_score = geno_57) %>%
eval_performance(
    phenotype,
    file.path(data_d, 'results', 'snpnet.lambda57.tsv'),
    family
)


phenotype_name,split,geno,covar,geno_covar,geno_delta,n_variables,case_n,control_n
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>
HC382,train,0.7598510,0.5387748,0.7588080,0.220033257,6564,31741,204250
HC382,test,0.6122056,0.5352619,0.6166757,0.081413781,6564,8934,58491
HC382,val,0.6149959,0.5281458,0.6185883,0.090442561,6564,4553,29160
HC382,non_british_white,0.6106912,0.5676678,0.6274338,0.059766007,6564,3243,21662
HC382,s_asian,0.5793712,0.5760666,0.6075215,0.031454854,6564,1171,6660
HC382,african,0.5535575,0.6095357,0.6164705,0.006934754,6564,871,5626
HC382,e_asian,0.5007736,0.5933069,0.5706519,-0.022654989,6564,166,1538


In [47]:
phe_score_intermediate_df %>%
select(-geno_score) %>%
rename(geno_score = geno_56) %>%
eval_performance(
    phenotype,
    file.path(data_d, 'results', 'snpnet.lambda56.tsv'),
    family
)


phenotype_name,split,geno,covar,geno_covar,geno_delta,n_variables,case_n,control_n
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>
HC382,train,0.7462311,0.5387748,0.7451270,0.206352251,5631,31741,204250
HC382,test,0.6122403,0.5352619,0.6166296,0.081367675,5631,8934,58491
HC382,val,0.6145204,0.5281458,0.6179748,0.089829013,5631,4553,29160
HC382,non_british_white,0.6101182,0.5676678,0.6270997,0.059431843,5631,3243,21662
HC382,s_asian,0.5796964,0.5760666,0.6076018,0.031535122,5631,1171,6660
HC382,african,0.5541298,0.6095357,0.6176065,0.008070819,5631,871,5626
HC382,e_asian,0.5041597,0.5933069,0.5749448,-0.018362135,5631,166,1538


In [40]:
eval_55_df

phenotype_name,split,geno,covar,geno_covar,geno_delta,n_variables,case_n,control_n
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>
HC382,train,0.7323389,0.5387748,0.7312488,0.19247405,4764,31741,204250
HC382,test,0.6120284,0.5352619,0.6163027,0.08104073,4764,8934,58491
HC382,val,0.6139289,0.5281458,0.6172363,0.08909049,4764,4553,29160
HC382,non_british_white,0.6093796,0.5676678,0.6265839,0.05891606,4764,3243,21662
HC382,s_asian,0.5801543,0.5760666,0.6076715,0.03160488,4764,1171,6660
HC382,african,0.5543122,0.6095357,0.6185394,0.00900363,4764,871,5626
HC382,e_asian,0.5086445,0.5933069,0.5790888,-0.01421812,4764,166,1538


In [42]:
roc_tests[['geno_56_vs_geno_55']]


	DeLong's test for two correlated ROC curves

data:  roc(sdf$phe, sdf[[sprintf("geno_%d", lambda_idx - 1)]], levels = c(control = 1,  and roc(sdf$phe, sdf[[sprintf("geno_%d", lambda_idx)]], levels = c(control = 1,     case = 2), direction = "<") and     case = 2), direction = "<")
Z = -1.6211, p-value = 0.105
alternative hypothesis: true difference in AUC is not equal to 0
sample estimates:
AUC of roc1 AUC of roc2 
  0.6139289   0.6145204 
